In [1]:
from glob import glob

files = glob('dumping-*.txt')
files

['dumping-instagram.txt',
 'dumping-parliament.txt',
 'dumping-twitter.txt',
 'dumping-cleaned-common-crawl.txt',
 'dumping-academia.txt',
 'dumping-iium.txt',
 'dumping-wiki.txt',
 'dumping-news.txt',
 'dumping-watpadd.txt',
 'dumping-pdf.txt',
 'dumping-common-crawl.txt']

In [2]:
with open('dumping-pdf.txt') as fopen:
    data = fopen.read().split('\n')

In [3]:
len(data)

651957

In [6]:
from tqdm import tqdm

def combined(data):
    results, result = [], []
    for i in tqdm(data):
        if not len(i) and len(result):
            results.append(result)
            result = []
        else:
            result.append(i)
    if len(result):
        results.append(result)
    return results

In [8]:
from unidecode import unidecode
import re

alphabets = '([A-Za-z])'
prefixes = (
    '(Mr|St|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|Mt|Puan|puan|Tuan|tuan|sir|Sir)[.]'
)
suffixes = '(Inc|Ltd|Jr|Sr|Co|Mo)'
starters = '(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever|Dia|Mereka|Tetapi|Kita|Itu|Ini|Dan|Kami|Beliau|Seri|Datuk|Dato|Datin|Tuan|Puan)'
acronyms = '([A-Z][.][A-Z][.](?:[A-Z][.])?)'
websites = '[.](com|net|org|io|gov|me|edu|my)'
another_websites = '(www|http|https)[.]'
digits = '([0-9])'
before_digits = '([Nn]o|[Nn]ombor|[Nn]umber|[Kk]e|=)'
month = '([Jj]an(?:uari)?|[Ff]eb(?:ruari)?|[Mm]a(?:c)?|[Aa]pr(?:il)?|Mei|[Jj]u(?:n)?|[Jj]ula(?:i)?|[Aa]ug(?:ust)?|[Ss]ept?(?:ember)?|[Oo]kt(?:ober)?|[Nn]ov(?:ember)?|[Dd]is(?:ember)?)'


def split_into_sentences(text, minimum_length = 5):
    text = text.replace('\x97', '\n')
    text = '. '.join([s for s in text.split('\n') if len(s)])
    text = text + '.'
    text = unidecode(text)
    text = ' ' + text + '  '
    text = text.replace('\n', ' ')
    text = re.sub(prefixes, '\\1<prd>', text)
    text = re.sub(websites, '<prd>\\1', text)
    text = re.sub(another_websites, '\\1<prd>', text)
    text = re.sub('[,][.]+', '<prd>', text)
    if '...' in text:
        text = text.replace('...', '<prd><prd><prd>')
    if 'Ph.D' in text:
        text = text.replace('Ph.D.', 'Ph<prd>D<prd>')
    text = re.sub('[.]\s*[,]', '<prd>,', text)
    text = re.sub(before_digits + '\s*[.]\s*' + digits, '\\1<prd>\\2', text)
    text = re.sub(month + '[.]\s*' + digits, '\\1<prd>\\2', text)
    text = re.sub('\s' + alphabets + '[.][ ]+', ' \\1<prd> ', text)
    text = re.sub(acronyms + ' ' + starters, '\\1<stop> \\2', text)
    text = re.sub(
        alphabets + '[.]' + alphabets + '[.]' + alphabets + '[.]',
        '\\1<prd>\\2<prd>\\3<prd>',
        text,
    )
    text = re.sub(
        alphabets + '[.]' + alphabets + '[.]', '\\1<prd>\\2<prd>', text
    )
    text = re.sub(' ' + suffixes + '[.][ ]+' + starters, ' \\1<stop> \\2', text)
    text = re.sub(' ' + suffixes + '[.]', ' \\1<prd>', text)
    text = re.sub(' ' + alphabets + '[.]', ' \\1<prd>', text)
    text = re.sub(digits + '[.]' + digits, '\\1<prd>\\2', text)
    if '”' in text:
        text = text.replace('.”', '”.')
    if '"' in text:
        text = text.replace('."', '".')
    if '!' in text:
        text = text.replace('!"', '"!')
    if '?' in text:
        text = text.replace('?"', '"?')
    text = text.replace('.', '.<stop>')
    text = text.replace('?', '?<stop>')
    text = text.replace('!', '!<stop>')
    text = text.replace('<prd>', '.')
    sentences = text.split('<stop>')
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences if len(s) > minimum_length]
    return sentences

split_into_sentences('733 ke . 633 , berlaku penurunan akibat kesan program PMI .')

['733 ke.633 , berlaku penurunan akibat kesan program PMI .']

In [9]:
import re

def strip(string):
    string = ' '.join(string)
    string = re.sub(r'[ ]+', ' ', string.replace('\n', ' ').replace('\t', ' ')).strip()
    return split_into_sentences(string)

In [11]:
results = combined(data)
len(results)

100%|██████████| 651957/651957 [00:00<00:00, 1546287.51it/s]


55538

In [13]:
output = []

for r in tqdm(results):
    output.extend(strip(r) + [''])

100%|██████████| 100/100 [00:00<00:00, 2912.06it/s]


In [15]:
with open('dumping-pdf.txt', 'w') as fopen:
    fopen.write('\n'.join(output))

In [19]:
with open('dumping-parliament.txt') as fopen:
    data = fopen.read().split('\n')

In [20]:
results = combined(data)
len(results)

100%|██████████| 960869/960869 [00:00<00:00, 1856840.93it/s]


70046

In [22]:
results[:100]

[['JAWAPAN-JAWAPAN BAGI PERTANYAAN-PERTANYAAN JAWAB',
  'LISAN YANG TIDAK DIJAWAB DI DALAM DEWAN'],
 ['NOTA : [ RUJUK PENYATA RASMI HARIAN ( HANSARD )]',
  'NURELFIRA BINTI MOHD DAUD',
  'CAWANGAN PERUNDANGAN',
  'PARLIMEN MALAYSIA'],
 ['NO SOALAN : 8',
  'PEMBERITAHU PERTANYAAN PEWAN RAKYAT',
  'PERTANYAAN LISAN',
  'DARIPADA LAKSAMANA PERTAMA MOHAMAD IMRAN',
  'BIN ABD HAMID ( B )',
  'Minta MENTERI PERTAHANAN menyatakan apakah respon dan reaksi',
  'Kementerian terhadap tuntutan lima ( 5 ) perkara oleh NGO PAHLAWAN .'],
 ['Tuan Yang di-Pertua ,',
  'Suka saya memaklumkan bahawa pertubuhan ini tidak berdaftar',
  'dengan Jabatan Hal Ehwal Veteran ATM ( JHEV ) di bawah Akta Veteran',
  "menyatakan bahawa ' tiada persatuan boleh menjalankan apa-apa aktiviti",
  'veteran melainkan jika persatuan itu didaftarkan sebagai suatu persatuan',
  "veteran di bawah akta ini '.",
  'Persatuan ini tidak pernah mengemukakan sebarang tuntutan rasmi',
  'kepada Kementerian Pertahanan . Pihak Kementer

In [23]:
output = []

for r in tqdm(results):
    output.extend(strip(r) + [''])

100%|██████████| 70046/70046 [00:24<00:00, 2910.76it/s]


In [24]:
output[:100]

['JAWAPAN-JAWAPAN BAGI PERTANYAAN-PERTANYAAN JAWAB LISAN YANG TIDAK DIJAWAB DI DALAM DEWAN.',
 '',
 'NOTA : [ RUJUK PENYATA RASMI HARIAN ( HANSARD )] NURELFIRA BINTI MOHD DAUD CAWANGAN PERUNDANGAN PARLIMEN MALAYSIA.',
 '',
 'NO SOALAN : 8 PEMBERITAHU PERTANYAAN PEWAN RAKYAT PERTANYAAN LISAN DARIPADA LAKSAMANA PERTAMA MOHAMAD IMRAN BIN ABD HAMID ( B ) Minta MENTERI PERTAHANAN menyatakan apakah respon dan reaksi Kementerian terhadap tuntutan lima ( 5 ) perkara oleh NGO PAHLAWAN .',
 '',
 "Tuan Yang di-Pertua , Suka saya memaklumkan bahawa pertubuhan ini tidak berdaftar dengan Jabatan Hal Ehwal Veteran ATM ( JHEV ) di bawah Akta Veteran menyatakan bahawa ' tiada persatuan boleh menjalankan apa-apa aktiviti veteran melainkan jika persatuan itu didaftarkan sebagai suatu persatuan veteran di bawah akta ini '.",
 'Persatuan ini tidak pernah mengemukakan sebarang tuntutan rasmi kepada Kementerian Pertahanan .',
 'Pihak Kementerian hanya mengetahui tuntutan yang dibuat oleh Pertubuhan Prihatin 

In [25]:
with open('dumping-parliament.txt', 'w') as fopen:
    fopen.write('\n'.join(output))